# Thesis Rework

# A) already generated chemical space in the /data folder

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../morphing_operations')
sys.path.append('.')

import osc_discovery.cheminformatics.cheminformatics_mutation as cheminformatics_helpers_v2
from osc_discovery.morphing_typical_OSC_design import operations_typical_osc_design as operations
from osc_discovery.cheminformatics.cheminformatics_structure_check import check_rules_fulfilled
from osc_discovery.cheminformatics.cheminformatics_symmetry import Symmetry
from osc_discovery.cheminformatics.cheminformatics_misc import flatten_list, ase2rdkit, ase2xyz

import time, os, pickle
import numpy as np
import pandas as pd
import multiprocessing as mp

import rdkit
print(rdkit.__version__)
from rdkit import rdBase,Chem
from rdkit.Chem import AllChem,Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [ ]:
path = os.getcwd() + '/data/'
file = 'df_chemical_space_chons_4rings.json'

In [ ]:
# Dataframe containing entire chemical space
p = pd.read_json(path+file, orient='split')

In [ ]:
p.head()

In [ ]:
# Pick a random molecule in the space and get its smile string
i = np.random.randint(315451)
# i = 171557 # simple molecule
catalyst_smi = p.iloc[i]['molecule_smiles']
# catalyst_smi = 'O=c1cc2c(no1)-c1ncccc1C2=C1C=CC=C1'
# catalyst_smi = 'c1cc[nH]c1'
# catalyst_smi = 'N(c1ccccc1)c2ccccc2'
p.iloc[i]

In [ ]:
Chem.AddHs(Chem.MolFromSmiles(catalyst_smi))

[i=17] Provides a good Case study of OH, O, OOH adsorption dynamics

OH adsorbs on site 2 causing substantial bending of entire catalyst

O adsorbs on sites 2 & 3, again causing bending

OOH however, preferentially adsorbs on site 3... i.e. it hops from site 2 to 3 during relaxation, and the substrate remains relatively flat as opposed to bent as in the previous adsorption configurations

If we sequentially form the subsequent adsorbates from the previous adsorbate configurations, I wonder what would happen?

#### 2 Ways of forming intermediates:

1. OH, OOH adsorb on the same site, with O adsorbing with an additional neighboring site
2. OH, OOH adsorb on neighboring sites, with O mediating their transition between sites..
In which case, maybe 2 carbons together could be considered an active site?

# ASE and xTB api

In [ ]:
from osc_discovery.descriptor_calculation.conformers import get_conformers_rdkit as get_conformers
from xtb.ase.calculator import XTB

import ase
from ase.optimize import LBFGS
from ase.optimize.minimahopping import MinimaHopping, MHPlot
from ase.optimize.basin import BasinHopping
from ase.calculators.emt import EMT
from ase.units import kB, Hartree
from ase.geometry.analysis import Analysis

from ase.constraints import FixAtoms, Hookean, FixBondLengths, FixBondLength, FixInternals

from ase.vibrations import Vibrations

from ase.thermochemistry import HarmonicThermo

from rdkit.Chem.Draw import SimilarityMaps

import matplotlib.pyplot as plt
from itertools import tee
from copy import deepcopy

In [ ]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [ ]:
### Define the water molecule

# SMILE string for water
mol_smi = 'O'

# RDKIT representation
mol_rdkit = Chem.AddHs(Chem.MolFromSmiles(mol_smi))

# ASE representation using the conformer generation code
mol_ase = get_conformers(mol_smi)[0]

In [ ]:
### Define the Hydrogen molecule
mol_ase = ase.Atoms('2H', positions=[(0., 0., 0.), (0., 0., 1.)])

In [ ]:
# Define Calculator object from xTB python api and assign it to ase atoms
T, P = 298, 101325
calc = XTB(method="GFN2-xTB", accuracy=0.2, electronic_temperature=T)
mol_ase.calc = calc

In [ ]:
# Constrain the Oxygen atom
constraint = FixAtoms(indices=[0]) #constrain the 0th atom
constraint = FixAtoms(mask=[atom == 'O' for atom in mol_ase.symbols]) #create a boolean mask with 1's in indicies where you want constr.
mol_ase.set_constraint(constraint)

# del mol_ase.constraints # Remove Constraints if necessary

In [ ]:
# Local Geometry Optimization
opt = LBFGS(mol_ase, trajectory='H2O.traj')
opt.run(fmax=0.0005)

In [ ]:
vib.clean()

In [ ]:
# Vibrational Analysis
vib = Vibrations(mol_ase)
vib.run()
vib.summary()

There are only 3N-6 vibrational modes present in a molecule... the imaginary freq. of H20 must be a computational artifact

In [ ]:
# Thermodynamic Quantities
num_modes = 3 * mol_ase.get_global_number_of_atoms() - 6

vib_energies = vib.get_energies()[-num_modes:]
pot_energy = mol_ase.get_potential_energy()

In [ ]:
thermo = HarmonicThermo(vib_energies=vib_energies, potentialenergy=pot_energy)

In [ ]:
thermo.get_helmholtz_energy(T)

## Calculate

In [ ]:
def single_point(molecule, method="GFN2-xTB", accuracy=0.2, electronic_temperature=298.15, relaxation=False, trajectory=None, fmax=0.05):
    
    # Method for remove prior calculator
    
    # Attach calculator
    calculator = XTB(method=method, accuracy=accuracy, electronic_temperature=electronic_temperature)
    molecule.calc = calculator
    
    # Locally optimize geometry
    if relaxation:
        optimizer = LBFGS(molecule, trajectory=trajectory, logfile=None)
        optimizer.run(fmax=fmax)
    
    # Calculate Energy (eV)
    E = molecule.get_potential_energy()
    
    return E

In [ ]:
# Define Relavent Species
# adsorbate = ase.Atoms('2H', positions=[(0., 0., 0.), (0., 0., 1.)])
adsorbate = ase.Atoms(symbols=['O', 'H'], positions=[(0., 0., 0.), (0., 0., 1.)])
adsorbate1 = ase.Atoms(symbols=['O'], positions=[(0., 0., 0.)])
adsorbate2 = ase.Atoms(symbols=['O', 'O', 'H'], positions=[(0., 0., 0.), (0., 0., 1.), (0., 1., 1.)])

substrate = get_conformers(catalyst_smi)[0]

In [ ]:
# Fix first atom to origin and relax
adsorbate.set_constraint([FixAtoms(indices=[0])])
adsorbate1.set_constraint([FixAtoms(indices=[0])])
adsorbate2.set_constraint([FixAtoms(indices=[0])])

# Relax species via a single point calculation
Eads = single_point(adsorbate, relaxation=True)
Eads1 = single_point(adsorbate1, relaxation=True)
Eads2 = single_point(adsorbate2, relaxation=True)

# Remove Constraints
del adsorbate.constraints, adsorbate1.constraints, adsorbate2.constraints

In [ ]:
# Equilibrium distances
r_eq_OO = adsorbate2.get_distance(0, 1)
r_eq_OH = adsorbate.get_distance(0, 1)

In [ ]:
Esub = single_point(substrate, relaxation=True, trajectory='opt.traj', fmax=0.005)

In [ ]:
# Relax system before computing bonds, otherwise you might get faulty bond designations
ana = Analysis(substrate)
bonds_original = ana.all_bonds[0]

total_num_nonHs = len(substrate) - substrate.get_chemical_symbols().count('H')

substrate.info['bonds'] = bonds_original
substrate.info['nonH_count'] = int(total_num_nonHs)

In [ ]:
unique_bonds = ana.unique_bonds[0]

pairwise_unique_bonds = []
for i, b in enumerate(unique_bonds):
    for v in b:
        pairwise_unique_bonds.append(sorted([i, v]))

## Vizualize negative charge density

In [ ]:
mol = Chem.AddHs(Chem.MolFromSmiles(catalyst_smi))
mol

In [ ]:
mol2 = ase2rdkit(substrate)
print(mol2.GetConformer(0).GetPositions())
mol2

In [ ]:
from rdkit.Chem.Draw import SimilarityMaps
import io
from PIL import Image

In [ ]:
def show_png(data):
    bio = io.BytesIO(data)
    img = Image.open(bio)
    return img

In [ ]:
static_chgs = substrate.get_charges()

In [ ]:
d = Draw.MolDraw2DCairo(400, 400)
SimilarityMaps.GetSimilarityMapFromWeights(mol2, list(static_chgs), draw2d=d)
d.FinishDrawing()
show_png(d.GetDrawingText())

## Brute Force Optimziation 1

In [ ]:
def build(a, s, site, constraints_list, f=1.5):
    ads = a.copy()
    sub = s.copy()
    
    # Active site position and indices of neighboring 2 atoms
    p = sub[site].position
    b = sub.get_distances(site, indices=[i for i in range(len(sub))]).argsort()[1:3]
    # b = sub.info[site]
    
    # Determine vector 'n' that defines a plane between active site and 2 neighboring atoms
    diff = sub[b].get_positions() - p
    cross = np.cross(diff[0], diff[1])
    n = cross / np.linalg.norm(cross)
    
    # Rotate vector definining O-H or O-O bond into vector 'n' defining plane
    if len(ads) > 1:
        ads.rotate(ads.positions[1], n) 
        
    # Translate adsorbate a height 'f' above position of active site
    ads.translate(p + n*f)
    
    # Form composite system and introduce any constraints
    composite = sub + ads
    composite.set_constraint(constraints_list)
    
    return composite

In [ ]:
def hopping_and_identity_checks(substrate_original, composite_relaxed, candidate_site):
    s, c = substrate_original.copy(), composite_relaxed.copy()
    
    ### Check the active site of adsorbate upon relaxation
    adsorb_len = len(c) - len(s)
    actual_sites = c.get_distances(-adsorb_len, indices=[i for i in range(len(s))]).argsort()
    
    if adsorb_len == 1:
        # If O adsorbate, is it bound within atleast an atom to the candidate active site
        cond1 = np.isin(candidate_site, actual_sites[:2])
    else:
        cond1 = (candidate_site == actual_sites[0])
    
    ### Check if identity of adsorbate has been preserved
    equilibrium_distances = [0.963, 1.302] # OH and OO bond distances
    threshold = 0.3 # Angstrom
    ads_dist = [composite_relaxed.get_distance(-i, -(i+1)) for i in range(1, adsorb_len)]
    cond2 = all([(abs(i - j) < threshold) for i, j in zip(equilibrium_distances, ads_dist)])
    
    ### Check if the identity of the substrate has been preserved upon relaxation & desorption
    del c.constraints, c[-adsorb_len:]
    bonding_c = Analysis(c).all_bonds[0] #indices in ascending order
    cond3 = (list(s.info.values()) == bonding_c)
    
    return cond1, cond2, cond3

In [ ]:
constraint_OH = Hookean(len(substrate), len(substrate)+1, rt=1.4, k=5) #constraint for O-H bond in OH
constraint_OOH_a = Hookean(len(substrate), len(substrate)+1, rt=1.79, k=5) #constraint for O-O bond in OOH (C-O bond params used)
constraint_OOH_b = Hookean(len(substrate)+1, len(substrate)+2, rt=1.4, k=5) #constraint for O-H bond in OOH

## Brute Force Optimziation 2

In [ ]:
def far_neighbor_indices(atom_index, sub, nonH_neighbors=12):
    # What is the average non_hydrogenic size of a given moeity thats added via the morphing operations
    nonH_nearest_neighbors = sub.get_distances(atom_index, indices=[range(sub.info['nonH_count'])]).argsort()
    return nonH_nearest_neighbors[nonH_neighbors+1:]

In [ ]:
def build_config(a, s, site, constraints_list, f=1.5):
    ads_perp, ads_between, ads_para, sub = a.copy(), a.copy(), a.copy(), s.copy()
    
    # Active site position and indices of neighboring 2 atoms
    p = sub[site].position
    b = sub.info['bonds'][site]
    
    if ((len(b) != 4) & (len(b) != 1)):
        # Alkane carbon (4) or Carbonyl Oxygen (1), likely not an active site
    
        # Determine vector 'n_perp' that defines a plane between active site and 2 neighboring atoms
        # as well as 'n_para' that sits within the plane but away from the 2 neighboring atoms
        diff = sub[b].get_positions() - p
        cross = np.cross(diff[0], diff[1])
        n_perp = cross / np.linalg.norm(cross)
        n_para = -(diff[0] + diff[1]) / np.linalg.norm(diff[0] + diff[1])

        # Rotate vector definining O-H or O-O bond into vector 'n' defining plane
        if len(a) > 1:
            ads_perp.rotate(ads_perp.positions[1], n_perp)
            #ads_between.rotate(ads_between.positions[1], n_perp)

        # Translate adsorbate a height 'f' above position of active site
        ads_perp.translate(p + n_perp*f)
        #ads_between.translate(p + n_perp*f)
        
        # Form composite system and introduce any constraints
        composite_perp = sub + ads_perp
        #composite_perp_neg = sub + ads_perp_neg
        
        composite_perp.set_constraint(constraints_list)
        #composite_perp_neg.set_constraint(constraints_list)
        
        # Alkyne carbons don't need an additional parallel orientation
        alkyne_condition = (sub.get_angle(b[0], site, b[1]) < 170)
        if (len(b) == 2) & alkyne_condition:
            # Possibility to orient parallel to the plane defined by 'n' and away from neighbors

            if len(a) > 1:
                ads_para.rotate(ads_para.positions[1], n_para)

            ads_para.translate(p + n_para*f)
            composite_para = sub + ads_para
            composite_para.set_constraint(constraints_list)
            
            return [composite_perp, composite_para]
        else:
            return [composite_perp]
        
    else:
        return []

In [ ]:
def site_identity_volatization_checks(substrate_original, composite_relaxed, volatization_threshold=2):
    s, c = substrate_original.copy(), composite_relaxed.copy()
    ads_indx = [indx for indx in range(len(s), len(c))] # adsorbate indices
    
    ### List of bond indices for each atom, arranged in ascending order. Create dict for easy processing
    bonds = Analysis(c).all_bonds[0]
    b = deepcopy(bonds)
    bonds_dict = dict(zip(range(len(b)), b))
    
    ### Check where the Oxygen in the adsorbate has bonded
    sites = [sub_indx for sub_indx in bonds[ads_indx[0]] if sub_indx not in ads_indx]
    
    ### Check if the identity of the substrate has been preserved upon relaxation & desorption
    # Remove the adsorbate from the bonds dict and compare the resulting dict to the original substrate bonds list
    # Look at what the adsorbate (a_i) is bonded to (v), and remove the adsorbate from v's bonds list
    # Then delete the adsorbate from the dict
    for a_i in ads_indx:
        for v in bonds_dict[a_i]:
            bonds_dict[v].remove(a_i)
        del bonds_dict[a_i]
    
    cond1 = (list(bonds_dict.values()) == substrate.info['bonds'])

    ### Check if identity of adsorbate has been preserved
    equilibrium_distances, threshold = [0.963, 1.302], 0.3 # OH and OO bond distances, threshold for bond-breakage
    ads_dist = [c.get_distance(*ai) for ai in pairwise(ads_indx)]
    ads_dist.reverse()
    cond2 = all([(abs(i - j) < threshold) for i, j in zip(equilibrium_distances, ads_dist)])
    
    ### Check if adsorbate is properly bound and has not volatalized
    # There is perhaps a faster way to do this using only the 'bonds' list
    # if len(sites) == 0, then volatization has occured. But the adsorbate could pick off a hydrogen
    min_dist = c.get_distances(ads_indx[0], indices=[i for i in range(s.info['nonH_count'])]).min()
    cond3 = (min_dist < volatization_threshold)
    
    return cond1, cond2, cond3, sites

In [ ]:
def volatization_and_identity_checks(substrate_original, composite_relaxed, volatization_threshold=2):
    s, c = substrate_original.copy(), composite_relaxed.copy()
    adsorb_len = len(c) - len(s)
    
    ### Check if adsorbate is properly bound and has not volatalized
    min_dist = c.get_distances(-adsorb_len, indices=[i for i in range(s.info['nonH_count'])]).min()
    cond1 = (min_dist < volatization_threshold)
    
    ### Check if identity of adsorbate has been preserved
    equilibrium_distances = [0.963, 1.302] # OH and OO bond distances
    threshold = 0.3 # Angstrom
    ads_dist = [composite_relaxed.get_distance(-i, -(i+1)) for i in range(1, adsorb_len)]
    cond2 = all([(abs(i - j) < threshold) for i, j in zip(equilibrium_distances, ads_dist)])
    
    ### Check if the identity of the substrate has been preserved upon relaxation & desorption
    del c.constraints, c[-adsorb_len:]
    bonding_c = Analysis(c).all_bonds[0] #indices in ascending order
    cond3 = (list(s.info.values()) == bonding_c)
    
    return cond1, cond2, cond3

In [ ]:
def plane(p1, p2, p3, offset):
    # These two vectors are in the plane
    v1 = p3 - p1
    v2 = p2 - p1

    # the cross product is a vector normal to the plane
    cp = np.cross(v1, v2)
    a, b, c = cp

    # This evaluates a * x + b * y + c * z + d = 0
    d = -np.dot(cp, p3)
    
    # Offset in angstrom to place plane above atoms
    d += offset
    
    return a, b, c, d

In [ ]:
def relax_configurations(configurations, method='GFN2-xTB'):
    start = time.time()
    for i, config in enumerate(configurations):
        print('Iteration:', i, '/', len(configurations))
        
        config.info['energy'] = single_point(config, method=method, relaxation=True, fmax=0.05, trajectory='opt{}.traj'.format(i))
        c1, c2, c3, active_sites = site_identity_volatization_checks(substrate, config)
        config.info['checks'] = all([c1, c2, c3])
        config.info['active sites'] = active_sites
        
        print('Energy (eV):', config.info['energy'])
        print('Checks:', config.info['checks'])
        print('Active Site:', config.info['active sites'])
        print('----------------------------------------')
    print('Elapsed Time:', time.time()-start)

In [ ]:
def enumerate_configuration_space(ads, sub, additional_constraints, fix_far_away=12):
    a, s = ads.copy(), sub.copy()
    
    ### Create initial configurations complete with desired constraints
    configs = []

    for i in range(s.info['nonH_count']):
        constr = [FixAtoms(indices=far_neighbor_indices(i, s, nonH_neighbors=fix_far_away))]
        constr += additional_constraints
        configs += build_config(a, s, i, constr)
    
    return configs

# Well-packaged

use copy.deepcopy() when forming copies, to avoid errors...

In [ ]:
os.chdir('run')

#### OH

In [ ]:
# Define Inputs
constraints = [Hookean(-len(adsorbate), -len(adsorbate)+1, rt=1.4, k=5)] #constraint for O-H bond in OH

# Run Workflow
configs = enumerate_configuration_space(adsorbate, substrate, constraints, fix_far_away=12)
relax_configurations(configs)
configurations = configs.copy()

# Calculate Params
E = np.array([config.info['energy'] for config in configurations])
Eadsorption = E - (Eads + Esub)

In [ ]:
[(j, site_identity_volatization_checks(substrate, config)) for j, config in enumerate(configs)]

In [ ]:
Eadsorption

#### O

In [ ]:
# Run Workflow
configs1 = enumerate_configuration_space(adsorbate1, substrate, [], fix_far_away=12)
relax_configurations(configs1)
configurations1 = configs1.copy()

# Calculate Params
E1 = np.array([config.info['Energy'] for config in configurations1])
Eadsorption1 = E1 - (Eads1 + Esub)

#### OOH

In [ ]:
# Define Inputs
constraint_OOH_a = Hookean(-len(adsorbate2), -len(adsorbate2)+1, rt=1.79, k=5) #constraint for O-O bond in OOH (C-O bond params used)
constraint_OOH_b = Hookean(-len(adsorbate2)+1, -len(adsorbate2)+2, rt=1.4, k=5) #constraint for O-H bond in OOH
constraints = [constraint_OOH_a, constraint_OOH_b]

# Run Workflow
configs2 = enumerate_configuration_space(adsorbate2, substrate, constraints, fix_far_away=12)
relax_configurations(configs2)
configurations2 = configs2.copy()

# Calculate Params
E2 = np.array([config.info['Energy'] for config in configurations2])
Eadsorption2 = E2 - (Eads2 + Esub)

#### Summary

In [ ]:
data = np.array([where_localized, where_localized1, where_localized2, Eadsorption, Eadsorption1, Eadsorption2])
col = ['where', 'where1', 'where2', 'Eads', 'Eads1', 'Eads2']
pd.DataFrame(data.T, columns=col)

s

In [ ]:
print(np.array([Eadsorption.argsort(), Eadsorption1.argsort(), Eadsorption2.argsort()]))

In [ ]:
d = get_min_site_adsorption_energy(where_localized, Eadsorption).values()
d2 = get_min_site_adsorption_energy(where_localized2, Eadsorption2).values()

In [ ]:
# Scaling Relations
plt.scatter(d, d2)

#### High Fidelity Scheme

In [ ]:
### Create initial configurations complete with desired constraints
configs = []

for i in range(total_num_nonHs):
    constr = [Hookean(len(substrate), len(substrate)+1, rt=1.4, k=5), FixAtoms(indices=far_neighbor_indices(i, substrate))]
    configs += build_config(adsorbate, substrate, i, constr)

In [ ]:
### Relax configurations
relax_configurations(configs)

In [ ]:
### Filter configurations for fidelity
# configurations = [config for config in configs if config.info['Fidelity Checks'] == True].copy()
configurations = configs.copy()

In [ ]:
E = np.array([config.info['Energy'] for config in configurations])
Checks = np.array([config.info['Fidelity Checks'] for config in configurations])
Eadsorption = (E - (Eads + Esub))

In [ ]:
where_localized = [where_adsorbate_location(substrate, config) for config in configurations]

In [ ]:
[(j, x, y) for j,(x,y) in enumerate(zip(where_localized, Eadsorption))]

#### O

In [ ]:
lowest_energy_config = configurations[Eadsorption.argmin()].copy()
configs_sorted = [y for _, y in sorted(zip(Eadsorption, configurations))]
checks_sorted = [y for _, y in sorted(zip(Eadsorption, Checks))]

In [ ]:
configs_rdkit_sorted = [ase2rdkit(config) for config in configs_sorted]

In [ ]:
l = ['{}, {}, {} eV'.format(i, c, e) for i, c, e in zip(Eadsorption.argsort(), checks_sorted, sorted(Eadsorption))]
Draw.MolsToGridImage(configs_rdkit_sorted, legends=l)

In [ ]:
[(j, x, y) for j,(x,y) in enumerate(zip(where_localized, Eadsorption))]

In [ ]:
where_localized = [where_adsorbate_location(substrate, config) for config in configurations]

In [ ]:
"""Multiprocessing doesnt realy work in Jupyter Notebook. Need to define function in a .py file, and call the multiprocessing
action in a separte script"""

# def relaxation_multiprocessor(l):
#     "Multiprocessing helper function"
#     return single_point(l, relaxation=True)

# start = time.time()
# multi_processors = 8 
# pool = mp.Pool(processes=multi_processors)
# Es_mp = pool.map(relaxation_multiprocessor, configs[:4])
# print(time.time() - start)

## OH Intermediate

#### Issues and Suggestions Revisited

2. Avoiding deprotonation of alkane hydrogens or skip them alltogether as active sites

Ex. For bond order = 4 atoms, you could form a plane including the hydrogens, that way the adsorbate is placed sufficiently far away and isnt able to be abstracted by the adsorbate

3. Avoiding volatalization... seems to be the case for OOH radical especially, specifically when it binds heteratoms. This situation seems not to be so stable.

Volatalization Check: Check if distance between adsorbate and closest substrate atom is large

It seems that sometimes, theres no way to avoid the adsorbate from relaxing on another neighboring active site. I think its more important that we simply note final adsorption site!

Also, the site for OH adsorption isnt always the same as for OOH!

In [ ]:
indx = []
E = []
checks_list = []

h = 1.5 #offset/height of adsorbate above substrate
fmax=0.05
i = 0

constr = [constraint_OH]

start = time.time()
total_num_nonHs = len([atom for atom in substrate if atom.symbol != 'H'])

for atom in substrate:
    if atom.symbol != 'H':
        # Build system
        comp = build(adsorbate, substrate, atom.index, constr, f=h)
        
        # Local Optimziation, Relax the composite system and determine energy
        Ecomposite = single_point(comp, relaxation=True, fmax=fmax, trajectory='opt/opt{}.traj'.format(atom.index))
        
        # Perform Checks
        checks = hopping_and_identity_checks(substrate, comp, atom.index)
        
        # Form lists
        indx.append(atom.index)
        E.append(Ecomposite)
        checks_list.append(checks)
        #OH_distance.append(comp.get_distance(-2, -1))
        #Osub_distance.append(comp.get_distance(atom.index, -2))
        
        # Output
        print("--------------------------------")
        print("Iteration: ", i, '/', total_num_nonHs-1)
        print("Active Site Persistence?: ", checks[0])
        print("Adsrbate Identity Preserved?: ", checks[1])
        print("Substrate Identity Preserved?: ", checks[2])
        i += 1

end = time.time()
print("Total Duration:", end - start, 'Seconds')

In [ ]:
E = np.array(E)
surf_E = E - (Eads + Esub)

In [ ]:
hetero_neighbor = []

for atom in substrate:
    if atom.symbol != 'H':
        neighbors = substrate[substrate.info[atom.index]].numbers
        hetero_neighbor.append(np.isin(neighbors, [7, 8, 16]).sum())
    
nonHs = [atom.symbol for atom in substrate if atom.symbol != 'H']

In [ ]:
E0 = pd.DataFrame(np.array([nonHs, hetero_neighbor, surf_E, checks_list]).T, index=indx, columns=['Site', 'Hetero Neighbors', 'Eads OH (eV)', 'Checks'])
E0

## O Intermediate

2. If it binds to to 2 carbons, we need to figure out what happens when OH is added to form OOH... do we still sit on the same active site, or do we hop active sites? Important!!!!

In [ ]:
indx = []
E = []
checks_list1 = []

h = 1.5 #offset/height of adsorbate above substrate
fmax=0.05
i = 0

constr = []

start = time.time()
total_num_nonHs = len([atom for atom in substrate if atom.symbol != 'H'])

for atom in substrate:
    if atom.symbol != 'H':
        # Build system
        comp = build(adsorbate1, substrate, atom.index, constr, f=h)
        
        # Local Optimziation, Relax the composite system and determine energy
        Ecomposite = single_point(comp, relaxation=True, fmax=fmax, trajectory='opt1/opt{}.traj'.format(atom.index))
        
        # Perform Checks
        checks = hopping_and_identity_checks(substrate, comp, atom.index)
        
        # Form lists
        indx.append(atom.index)
        E.append(Ecomposite)
        checks_list1.append(checks)
        
        # Output
        print("--------------------------------")
        print("Iteration: ", i, '/', total_num_nonHs-1)
        print("Active Site Persistence?: ", checks[0])
        print("Adsrbate Identity Preserved?: ", checks[1])
        print("Substrate Identity Preserved?: ", checks[2])
        i += 1

end = time.time()
print("Total Duration:", end - start, 'Seconds')

In [ ]:
E = np.array(E)
surf_E1 = E - (Eads1 + Esub)

In [ ]:
E1 = pd.DataFrame(np.array([nonHs, hetero_neighbor, surf_E1, checks_list1]).T, index=indx, columns=['Site', 'Hetero Neighbors', 'Eads OH (eV)', 'Checks'])
E1

### OOH Intermediate

1. Weak binding to heteroatoms...usually leads to hopping

In [ ]:
indx = []
E = []
checks_list2 = []

h = 1.5 #offset/height of adsorbate above substrate
fmax=0.05
i = 0

constr = [constraint_OOH_a, constraint_OOH_b]

start = time.time()
total_num_nonHs = len([atom for atom in substrate if atom.symbol != 'H'])

for atom in substrate:
    if atom.symbol != 'H':
        # Build system
        comp = build(adsorbate2, substrate, atom.index, constr, f=h)
        
        # Local Optimziation, Relax the composite system and determine energy
        Ecomposite = single_point(comp, relaxation=True, fmax=fmax, trajectory='opt2/opt{}.traj'.format(atom.index))
        
        # Perform Checks
        checks = hopping_and_identity_checks(substrate, comp, atom.index)
        
        # Form lists
        indx.append(atom.index)
        E.append(Ecomposite)
        checks_list2.append(checks)
        
        # Output
        print("--------------------------------")
        print("Iteration: ", i, '/', total_num_nonHs-1)
        print("Active Site Persistence?: ", checks[0])
        print("Adsrbate Identity Preserved?: ", checks[1])
        print("Substrate Identity Preserved?: ", checks[2])
        i += 1
        break

end = time.time()
print("Total Duration:", end - start, 'Seconds')

In [ ]:
E = np.array(E)
surf_E2 = E - (Eads2 + Esub)

In [ ]:
E2 = pd.DataFrame(np.array([nonHs, hetero_neighbor, surf_E2, checks_list2]).T, index=indx, columns=['Site', 'Hetero Neighbors', 'Eads OH (eV)', 'Checks'])
E2

### Intermediate Summary

In [ ]:
data = np.array([nonHs, hetero_neighbor, surf_E, surf_E1, surf_E2, checks_list, checks_list1, checks_list2]).T
col = ['Site', 'Hetero Neighbors', 'ΔEads OH (eV)', 'ΔEads O (eV)', 'ΔEads OOH (eV)', 'Checks', 'Checks1', 'Checks2']
adsorption_energy_summary = pd.DataFrame(data, index=indx, columns=col)
adsorption_energy_summary

In [ ]:
plt.scatter(adsorption_energy_summary['ΔEads OH (eV)'], adsorption_energy_summary['ΔEads OOH (eV)'])
plt.ylabel('OOH Ads Energy')
plt.xlabel('OH Ads Energy')
plt.title('Scaling Relations!')

Discrepencies in scaling relations probably due to hopping of adsorbates/changes in substrate structure

# Global Optimization

1. Relax individual adsorbate and substrate geometries
2. Fix substrate position and apply hookean force between adsorbate atoms to preserve their identities
3. Relax this composite system using global opt. algorithm
4. Remove substrate constraints, and further relax the entire system

Minima Hopping

In [ ]:
single_point(out)

In [ ]:
# Gloabl Optimization
opt = MinimaHopping(out,
                    Ediff0=2.5,
                    T0=2000.)

opt(totalsteps=3)

In [ ]:
mhplot = MHPlot()
mhplot.save_figure('summary.png')

Basin Hopping

In [ ]:
single_point(out)

In [ ]:
opt = BasinHopping(atoms=out,         # the system to optimize
                  temperature=100 * kB, # 'temperature' to overcome barriers
                  dr=0.5,               # maximal stepwidth
                  optimizer=LBFGS,      # optimizer to find local minima
                  fmax=0.1,
                  trajectory='opt.traj')

opt.run(steps=10)

# B) generate_chemical_space.py in scripts folder

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../morphing_operations')
sys.path.append('.')

import osc_discovery.cheminformatics.cheminformatics_mutation as cheminformatics_helpers_v2
from osc_discovery.morphing_typical_OSC_design import operations_typical_osc_design as operations
from osc_discovery.cheminformatics.cheminformatics_structure_check import check_rules_fulfilled
from osc_discovery.cheminformatics.cheminformatics_symmetry import Symmetry
from osc_discovery.cheminformatics.cheminformatics_misc import flatten_list

import time, os, pickle
import numpy as np
import pandas as pd
import multiprocessing as mp

import rdkit
print(rdkit.__version__)
from rdkit import rdBase,Chem
from rdkit.Chem import AllChem,Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [ ]:
''' Script to enumerate full chemical space within bounds on molecular size '''

# Check 
failed = 0
stopped = 0
new_generation = 0
allow_only_graph_symmetric = True
multi_proc = 24

cheminformatics_helpers_v2.operations = operations
cheminformatics_helpers_v2.failed = 0
cheminformatics_helpers_v2.stopped = 0
cheminformatics_helpers_v2.new_generation = 0
cheminformatics_helpers_v2.allow_only_graph_symmetric = True
cheminformatics_helpers_v2.multi_proc = multi_proc
cheminformatics_helpers_v2.preset_structure_check='test_osc_space'
cheminformatics_helpers_v2.only_use_brute_force_code=False

mol_test=Chem.MolFromSmiles('c1ccccc1')
check_rules_fulfilled(mol_test, '',  cheminformatics_helpers_v2.preset_structure_check ,verbose=True)

In [ ]:
mol_test

In [ ]:
def run_generation_text_based(textfile=''):
        
    template_save='{}__{}__{}'
    dict_smi_list_mutate={}
    
    print(textfile)   
 
    # generate a uniqe dict
    if textfile=='':
        dict_smi_list_mutate['c1ccccc1']='0__none__none'
        cheminformatics_helpers_v2.last_generation=0
        with open('gen__0.smi','w') as out:
            out.write('c1ccccc1 0__none__none')
            
    else:
        cheminformatics_helpers_v2.last_generation = int(textfile.split('__')[-1].split('.smi')[0])       
        i=0
        dict_smi_list_file={}
        with open(textfile) as out:
            for i,line in enumerate(out.readlines()):
                line=line.split()
                dict_smi_list_file[line[0]]=line[1]

            print('Number of lines: {}'.format(i))

        already_mutated=[]
        for i in range(cheminformatics_helpers_v2.last_generation):
            filex='gen__{}.smi'.format(i)
            print(filex)
            with open(filex) as out:
                for line in out.readlines():
                    already_mutated.append(line.split()[0])
                    
        to_mutate = set(dict_smi_list_file.keys()) - set(already_mutated)
        for m in list(to_mutate):
            dict_smi_list_mutate[m]=dict_smi_list_file[m]
                    
    print('Unique molecules to mutate: {}'.format(len(dict_smi_list_mutate.keys())))
    
    cheminformatics_helpers_v2.new_generation = cheminformatics_helpers_v2.last_generation + 1
    cheminformatics_helpers_v2.mols_last_generation_smiles = list(dict_smi_list_mutate.keys())
    textfile_new_gen='gen__{}.smi'.format(cheminformatics_helpers_v2.new_generation) 
    
    pool = mp.Pool(processes=multi_proc)
    new_frames = pool.map(cheminformatics_helpers_v2.run_mutation, list(dict_smi_list_mutate.keys()) )
    pool.close()

    new_frames_scratch=[]
    for framex in new_frames:
        for frame in framex:
            new_frames_scratch.append(frame)
    new_frames=new_frames_scratch
    
    ##new_frames=list(flatten_list(new_frames))
  
    with open(textfile_new_gen, 'w') as out:
        for i,f in enumerate(new_frames):
            if len(f.molecule_smiles.values[0])==0: continue
            # format: smi gen_nr__mol_smi_lastgen__op
            out.write('{} {}__{}__{}\n'.format(f.molecule_smiles.values[0], cheminformatics_helpers_v2.new_generation, 
                                           f.molecule_last_gen.values[0], f.operation.values[0]))
    
    statistics = [len(new_frames), cheminformatics_helpers_v2.stopped, cheminformatics_helpers_v2.failed]
    return statistics, textfile_new_gen

In [ ]:
if __name__=='__main__':

    time_start_total=time.time()
    
    ratios=[]
    n_mols_unique=[]
    #fname=sys.argv[1]
    fname=''
    
    for i in range(2):
    
        time_start=time.time()
        statistics, fname = run_generation_text_based(fname)
        print('Time for step: {}'.format(time.time()-time_start))
        print('New lines: {}, Stopped: {}, failed: {}'.format(*statistics))
        print('')
        print('')
        
    print('Total time: {}'.format(time.time()-time_start_total))